In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! apt install liblz4-tool

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  liblz4-1
The following NEW packages will be installed:
  liblz4-tool
The following packages will be upgraded:
  liblz4-1
1 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 124 kB of archives.
After this operation, 380 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 liblz4-1 amd64 0.0~r131-2ubuntu3.1 [48.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 liblz4-tool amd64 0.0~r131-2ubuntu3.1 [75.0 kB]
Fetched 124 kB in 2s (74.1 kB/s)
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../liblz4-1_0.0~r131-2ubuntu3.1_amd64.deb ...
Unpacking liblz4-1:amd64 (0.0~r131-2ubuntu3.1) over (0.0~r131-2ubuntu3) ...
Setting up liblz4-1:amd64 (0.0~r131-2ubuntu3.1) ...
Selecting previously unselected pack

In [ ]:
! lz4 -d  "/content/drive/MyDrive/NLP/Data/yelp_data.tar.lz4" /content/yelp_data.tar


Successfully decoded 11375093760 bytes 


In [ ]:
! tar -xvf yelp_data.tar 

data/
data/yelp_academic_dataset_tip_00
data/yelp_academic_dataset_tip_01
data/yelp_academic_dataset_tip_02
data/yelp_academic_dataset_tip_03
data/yelp_academic_dataset_tip_04
data/yelp_academic_dataset_tip_05
data/yelp_academic_dataset_tip_06
data/yelp_academic_dataset_tip_07
data/yelp_academic_dataset_tip_08
data/yelp_academic_dataset_tip_09
data/yelp_academic_dataset_tip_10
data/yelp_academic_dataset_tip_11
data/yelp_academic_dataset_tip_12
data/yelp_academic_dataset_tip_13
data/yelp_academic_dataset_tip_14
data/yelp_academic_dataset_tip_15
data/yelp_academic_dataset_tip_16
data/yelp_academic_dataset_tip_17
data/yelp_academic_dataset_tip_18
data/yelp_academic_dataset_tip_19
data/yelp_academic_dataset_tip_20
data/yelp_academic_dataset_tip_21
data/yelp_academic_dataset_tip_22
data/yelp_academic_dataset_tip_23
data/yelp_academic_dataset_tip_24
data/yelp_academic_dataset_tip_25
data/yelp_academic_dataset_tip_26
data/yelp_academic_dataset_tip_27
data/yelp_academic_dataset_tip_28
data/yel

In [ ]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import json
import copy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
# Load yelp_academic_dataset_review_00 
review_dataset = {}
for i in range(100):
	try:
		if i < 90:
			with open('data/yelp_academic_dataset_review_{}'.format(str(i).zfill(2)),encoding='utf-8') as f:
				for line in f:
					data = json.loads(line)
					review_dataset[data['business_id']] = data
		else:
			with open('data/yelp_academic_dataset_review_9{}'.format(str(i - 90).zfill(3)),encoding='utf-8') as f:
				for line in f:
					data = json.loads(line)
					review_dataset[data['business_id']] = data
	except:
		print('File not found {}'.format(i))
		break

In [ ]:
print(len(review_dataset))

28085


In [ ]:
# Load yelp_academic_dataset_business_00
business_dataset = {}
for i in range(20):
	try:
		with open('data/yelp_academic_dataset_business_{}'.format(str(i).zfill(2))) as f:
			for line in f:
				data = json.loads(line)
				business_dataset[data['business_id']] = data
	except:
		print("File not found {}".format(i))
		break


File not found 17


In [ ]:
# Find the match between review and business_dataset
# Copy review_dataset to merged_dataset
merged_dataset = {}
for review in review_dataset:
	if review in business_dataset:
# 		print(review)
# 		print(review_dataset[review])
# 		print(business_dataset[review])
# 		print()
		# Merge the two datasets
		if review not in merged_dataset:
			merged_dataset[review] = [copy.deepcopy(review_dataset[review])]
			merged_dataset[review].append(copy.deepcopy(business_dataset[review]))
		else: 
			merged_dataset[review].append(copy.deepcopy(review_dataset[review]))
			merged_dataset[review].append(copy.deepcopy(business_dataset[review]))

In [ ]:
# filter out all review with less than a usefulness of 3
keys_to_delete = []
for i in merged_dataset:
	if merged_dataset[i][0]['useful'] < 3:
		# remove the review
		keys_to_delete.append(i)
RestaurantsTakeOut = []
RestaurantsDelivery = []
GoodForMeal_dessert = []
GoodForMeal_latenight = []
GoodForMeal_lunch = []
GoodForMeal_dinner = []
GoodForMeal_brunch = []
GoodForMeal_breakfast = []
keys_to_delete = []
for i in merged_dataset:
	try:
		# Get simple features
		RestaurantsTakeOut_tmp = merged_dataset[i][1]['attributes']["RestaurantsTakeOut"]
		RestaurantsDelivery_tmp = merged_dataset[i][1]['attributes']["RestaurantsDelivery"]
		

		# Get the complex features
		GoodForMeal_tmp = merged_dataset[i][1]['attributes']["GoodForMeal"]
		

		# Find and replace False with "False" in tmp_string
		GoodForMeal_tmp = GoodForMeal_tmp.replace("False", "'False'")
		# Do the same for True
		GoodForMeal_tmp = GoodForMeal_tmp.replace("True", "'True'")
		GoodForMeal_tmp = GoodForMeal_tmp.replace("'", '"')
		tmp_meal = json.loads(GoodForMeal_tmp)

		# Append the features to the lists
		GoodForMeal_dessert.append(tmp_meal['dessert'])
		GoodForMeal_latenight.append(tmp_meal['latenight'])
		GoodForMeal_lunch.append(tmp_meal['lunch'])
		GoodForMeal_dinner.append(tmp_meal['dinner'])
		GoodForMeal_brunch.append(tmp_meal['brunch'])
		GoodForMeal_breakfast.append(tmp_meal['breakfast'])
		RestaurantsTakeOut.append(RestaurantsTakeOut_tmp)
		RestaurantsDelivery.append(RestaurantsDelivery_tmp)
	# print(merged_dataset[i][1]['attributes']['BusinessParking']['Garage'])
	except:
		keys_to_delete.append(i)
	# 	pass
for i in keys_to_delete:
	merged_dataset.pop(i)

In [ ]:
print(len(merged_dataset))
print(len(GoodForMeal_dessert))
print(len(GoodForMeal_latenight))
print(len(GoodForMeal_lunch))
print(len(GoodForMeal_dinner))
print(len(GoodForMeal_brunch))
print(len(GoodForMeal_breakfast))
print(len(RestaurantsTakeOut))
print(len(RestaurantsDelivery))

4300
4300
4300
4300
4300
4300
4300
4300
4300


In [ ]:
print(json.dumps(merged_dataset, indent=4))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
review_keys = list(merged_dataset.keys())

In [ ]:
merged_dataset[review_keys[0]][0]['text']

"Love getting meat straight from a local butcher/business. They have a great selection to choose from. On this trip we ordered takeout, and munched on lunch that included the burger, chicken sandwich and fries. \n\nChicken was very simple and that's why it was so good - juicy and crispy. A burger can be defined by the bun and that is the case here as it comes on a tasty French roll. If the burger was cooked on an actual grill instead of a flat top, then I'd give this five stars."

In [ ]:
import nltk
nltk.download('stopwords')
#remove punctuation
import string
table = str.maketrans('', '', string.punctuation)

# Remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('wordnet')
# review_text = []
# for i in reviews_text_raw:
# 	review_text.append(' '.join([word for word in i.lower().split() if word not in stop]))
for i in review_keys:
    review_text = merged_dataset[i][0]['text']
    review_text = ' '.join([word for word in review_text.lower().split() if word not in stop]) #remove stopwords
    review_text = review_text.translate(table) #remove punc.
    review_text = ''.join(c for c in review_text if not c.isdigit()) #remove numbers
    review_text = review_text.replace('  ', ' ') #remove double spaces
    review_text = ' '.join([lemmatizer.lemmatize(word) for word in review_text.split()]) #lemminize
    merged_dataset[i][0]['text'] = review_text
    

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
print(merged_dataset[review_keys[0]][0]['text'])

love getting meat straight local butcherbusiness great selection choose from trip ordered takeout munched lunch included burger chicken sandwich fry chicken simple thats good juicy crispy burger defined bun case come tasty french roll burger cooked actual grill instead flat top id give five star


In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
for i in review_keys:
    single_review_vector = []
    review_text = merged_dataset[i][0]['text']
    for word in review_text.split():
        try:
            single_review_vector.append(np.asarray(wv[word]).astype('float32'))
#             single_review_vector.append(list(wv[word]))
        except:
            pass
            # print(f'Word {word} not in vocabulary')
            #we are ignoring words that are just combined by accident: fix later???
#     merged_dataset[i][0]['vec_review'] = np.asarray(single_review_vector).astype('float32')
    merged_dataset[i][0]['vec_review'] = single_review_vector
#     merged_dataset[i][0]['vec_review'] = np.array(single_review_vector, dtype=np.float)

In [ ]:
review_df = pd.DataFrame()

In [ ]:
for i in review_keys:
    review_df = review_df.append(pd.Series(merged_dataset[i][0]['vec_review']), ignore_index=True)

In [ ]:
for col in review_df.columns:
    indcies = [ i for i in range(0,len(review_df)) if type(review_df.loc[i, col]) == float]
    for i in indcies:
        review_df.at[i, col] = np.array(list(np.zeros(300)), dtype=np.float)


In [ ]:
# Convert from string False or True to 0 or 1
GoodForMeal_dessert = [0 if i == 'False' else 1 for i in GoodForMeal_dessert]
GoodForMeal_latenight = [0 if i == 'False' else 1 for i in GoodForMeal_latenight]
GoodForMeal_lunch = [0 if i == 'False' else 1 for i in GoodForMeal_lunch]
GoodForMeal_dinner = [0 if i == 'False' else 1 for i in GoodForMeal_dinner]
GoodForMeal_brunch = [0 if i == 'False' else 1 for i in GoodForMeal_brunch]
GoodForMeal_breakfast = [0 if i == 'False' else 1 for i in GoodForMeal_breakfast]
RestaurantsTakeOut = [0 if i == 'False' else 1 for i in RestaurantsTakeOut]
RestaurantsDelivery = [0 if i == 'False' else 1 for i in RestaurantsDelivery]

In [ ]:
merged_output = []
for i in range(len(GoodForMeal_dessert)):
    merged_output.append([GoodForMeal_dessert[i],GoodForMeal_latenight[i], GoodForMeal_lunch[i], GoodForMeal_dinner[i], GoodForMeal_brunch[i], GoodForMeal_breakfast[i], RestaurantsTakeOut[i], RestaurantsDelivery[i]])


In [ ]:
# Split the review vector and garage tensors into training and testing sets
from sklearn.model_selection import train_test_split
review_vector_train, review_vector_test, x_train, y_test = train_test_split(review_df, merged_output, test_size=0.2)

In [ ]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
# tf.transpose(review_vector_test_tens, [1, 0, 2])
review_vector_train_tens = tf.transpose(tf.constant(([review_vector_train[i] for i in review_vector_train.columns])), [1,0,2])
review_vector_test_tens = tf.transpose(tf.constant(([review_vector_test[i] for i in review_vector_test.columns])), [1,0,2])

In [ ]:
# Train lstm model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping


In [ ]:
from tensorflow.keras.layers import Attention

In [ ]:
from tensorflow.keras import optimizers as opt

In [ ]:
#Initialize the model
model = Sequential()
# Add an LSTM layer with 100 units
model.add(LSTM(100, input_shape=(review_vector_train_tens.shape[1], review_vector_train_tens.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
# Add an LSTM layer with 100 units
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
# Add an LSTM layer with 100 units
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
# Add an LSTM layer with 100 units
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
# Output layer
model.add(Dense(8, activation='sigmoid'))

# Train the model

model.compile(loss='binary_crossentropy', optimizer= opt.SGD(lr=0.01) , metrics=['accuracy']) #opt.SGD(lr=0.01)
# early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(review_vector_train_tens, x_train, epochs=10, batch_size=64, validation_split=0.2)

ValueError: ignored

In [ ]:
preds = model.predict(review_vector_test_tens)
preds = np.around(preds, decimals=0)

In [ ]:
preds.shape

(860, 8)

In [ ]:
y_test.shape

TensorShape([860, 8])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = []
for i in range(0,8):
  accuracy_val  = accuracy_score(preds[:,i], np.array(y_test)[:,i])
  acc.append(accuracy_val)

In [ ]:
np.mean(acc)

0.7754360465116279

In [ ]:
for i in acc:
  print(round(i, 4))

0.9419
0.9326
0.5174
0.4488
0.9128
0.9012
0.9605
0.5884
